# 场景一：角色扮演的写作

In [3]:
!ollama pull llama2-chinese:13b

pulling manifest
pulling 8359bebea988... 100% |██████████████████| (7.4/7.4 GB, 16 TB/s)        
pulling 65c6ec5c6ff0... 100% |████████████████████| (45/45 B, 1.8 MB/s)        
pulling dd36891f03a0... 100% |████████████████████| (31/31 B, 1.3 MB/s)        
pulling f94f529485e6... 100% |███████████████████| (382/382 B, 17 MB/s)        
verifying sha256 digest
writing manifest
removing any unused layers
success


In [ ]:
%pip install langchain langchain-core langchain-community

In [42]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama

# 设定系统上下文，构建提示词
template = """请扮演一位资深的技术博主，您将负责为用户生成适合在微博发送的中文帖文。
请把用户输入的内容扩展成 140 字左右的文字，并加上适当的 emoji 使内容引人入胜并专业。"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

# 通过 Ollama 加载 Llama 2 13B 对话补全模型
model = ChatOllama(model="llama2-chinese:13b")

# 通过 LCEL 构建调用链并执行得到文本输出
chain = prompt | model | StrOutputParser()
chain.invoke({ "input": "给大家推荐一本新书《LangChain实战》，让我们一起开始来学习 LangChain 吧！"})


'👋祝大家好！我今天特地为大家推荐了一本新书《LangChain实战》，让我们一起开始来学习 LangChain 吧！📚💻这本书是由一些专业的技术人员编写的，内容十分透彻、实用。🤝如果大家想要提高自己的编程水平，或者了解更多的开发框架，这本书一定会对你有所帮助。👍赞！\n'

In [47]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

prompt = ChatPromptTemplate.from_template("请编写一篇关于{topic}的中文小故事，不超过100字")
model = ChatOllama(model="llama2-chinese:13b")

chain = prompt | model
chain.invoke({"topic": "小白兔"})

AIMessage(content='\n小白兔在花草间跃步而来。看到了一颗美味的草根，便用长长的长着想吃下去。却发现这只不过是一个似人像的玩物。小白兔开始笑起来，感到十分惊喜。他懂得如何享受生活中的美好事物。\n')

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="rabbit")

'Tell me a funny joke about rabbit.'

In [11]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)
chat_template.format_messages(name="Bob", user_input="What is your name?")

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='What is your name?')]

In [17]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 创建一些反义词输入输出的示例内容
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    examples=examples, 
    example_prompt=example_prompt, 
    # 设定期望的示例文本长度
    max_length=25
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    # 设置示例以外部分的前置文本
    prefix="Give the antonym of every input",
    # 设置示例以外部分的后置文本
    suffix="Input: {adjective}\nOutput:\n\n",
    input_variables=["adjective"],
)

# 当用户输入的内容比较少时，所有示例都足够被使用
print(dynamic_prompt.format(adjective="big"))

# 当用户输入的内容足够长时，只有少量示例会被引用
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output:


Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:




In [35]:
from typing import List

from langchain_core.prompts import PromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field

class Actor(BaseModel):
    name: str = Field(description="name of an author")
    book_names: List[str] = Field(description="list of names of book they wrote")


actor_query = "随机生成一位知名的作家及其代表作品"

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="请回答下面的问题：\n{query}\n\n{format_instructions}\n如果输出是代码块，请不要包含首尾的```符号",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

input = prompt.format_prompt(query=actor_query)
print(input)

model = Ollama(model="llama2-chinese:13b")
output = model(input.to_string())

print(output)
parser.parse(output)

text='请回答下面的问题：\n随机生成一位知名的作家及其代表作品\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of an author", "type": "string"}, "book_names": {"title": "Book Names", "description": "list of names of book they wrote", "type": "array", "items": {"type": "string"}}}, "required": ["name", "book_names"]}\n```\n如果输出是代码块，请不要包含首尾的```符号'
{
    "name": "J.K. Rowling",
    "book_names": [
        "Harry Potter and the Philosopher's Stone", 
        "Harry Potter and the Chamber of Secrets", 
        "Harry Potter and the Prisoner of Azkaban

Actor(name='J.K. Rowling', book_names=["Harry Potter and the Philosopher's Stone", 'Harry Potter and the Chamber of Secrets', 'Harry Potter and the Prisoner of Azkaban', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Order of the Phoenix', 'Harry Potter and the Half-Blood Prince', 'Harry Potter and the Deathly Hallows'])